In [0]:
SELECT  *
FROM    (
            SELECT  i_manufact_id
                    ,SUM(ss_sales_price) sum_sales
                    ,AVG(SUM(ss_sales_price)) OVER (PARTITION BY i_manufact_id ) avg_quarterly_sales
            FROM workspace.tpcds_sf1000_iceberg.item
            ,       workspace.tpcds_sf1000_iceberg.store_sales
            ,       workspace.tpcds_sf1000_iceberg.date_dim
            ,       workspace.tpcds_sf1000_iceberg.store
            WHERE   ss_item_sk = i_item_sk
            AND     ss_sold_date_sk = d_date_sk
            AND     ss_store_sk = s_store_sk
            AND     d_month_seq IN (1219,1219 + 1,1219 + 2,1219 + 3,1219 + 4,1219 + 5,1219 + 6,1219 + 7,1219 + 8,1219 + 9,1219 + 10,1219 + 11)
            AND     (
                        (
                                    i_category IN ('Books','Children','Electronics')
                                    AND     i_class IN ('personal','portable','reference','self-help')
                                    AND     i_brand IN ('scholaramalgamalg #14','scholaramalgamalg #7','exportiunivamalg #9','scholaramalgamalg #9')
                        )
                        OR      (
                                    i_category IN ('Women','Music','Men')
                                    AND     i_class IN ('accessories','classical','fragrances','pants')
                                    AND     i_brand IN ('amalgimporto #1','edu packscholar #1','exportiimporto #1','importoamalg #1')
                        )
            )
            GROUP BY i_manufact_id
                      ,d_qoy
        ) tmp1
WHERE   CASE   WHEN avg_quarterly_sales > 0 THEN abs(sum_sales - avg_quarterly_sales) / avg_quarterly_sales
        ELSE NULL
END > 0.1
ORDER BY avg_quarterly_sales,sum_sales,i_manufact_id
LIMIT   100
;